In [1]:
pip install mysql-connector-python boto3 pandas


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import mysql.connector
import boto3
import pandas as pd
from io import StringIO

# AWS S3 Configuration
s3 = boto3.client('s3')
bucket_name = "fraud-detection-data-warehouse"
file_name = "fraud_test_latest.csv"

# Fetch the file from S3
response = s3.get_object(Bucket=bucket_name, Key=file_name)
csv_data = response['Body'].read().decode('utf-8')

# Convert CSV to DataFrame
df = pd.read_csv(StringIO(csv_data))


# Convert dob to MySQL DATE format (YYYY-MM-DD)
df['dob'] = pd.to_datetime(df['dob'], format='%d/%m/%Y').dt.date  # Keep only date, remove time part

# Print first few rows to verify changes
print(df[['dob']].head())

# Convert all numeric columns to native Python types
df = df.astype(object)

# Connect to MySQL
connection = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="root@123",
    database="frauddetectionDB"
)
cursor = connection.cursor()

# SQL Insert Query
insert_query = """
INSERT INTO transactions_One
(cc_num, merchant, category, amt, first, last, gender, street, city, state, zip, lat, longtitude, city_pop, job, dob, trans_num, unix_time, merch_lat, merch_long, is_fraud)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

df = df[['cc_num', 'merchant', 'category', 'amt', 'first', 'last', 
         'gender', 'street', 'city', 'state', 'zip', 'lat', 'longtitude', 'city_pop', 'job', 
         'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud']]

# Convert DataFrame to List of Tuples
data = [tuple(row) for row in df.itertuples(index=False, name=None)]

#Insert data in batches of 2000 rows
batch_size = 2000
for i in range(0, len(data), batch_size):
    batch = data[i:i + batch_size]
    cursor.executemany(insert_query, batch)
    connection.commit()
    print(f"Inserted {i + len(batch)} rows...")

print("Data insertion completed.")

# Close the connection
cursor.close()
connection.close()


          dob
0  1968-03-19
1  1990-01-17
2  1970-10-21
3  1987-07-25
4  1955-07-06
Inserted 2000 rows...
Inserted 4000 rows...
Inserted 6000 rows...
Inserted 8000 rows...
Inserted 10000 rows...
Inserted 12000 rows...
Inserted 14000 rows...
Inserted 16000 rows...
Inserted 18000 rows...
Inserted 20000 rows...
Inserted 22000 rows...
Inserted 24000 rows...
Inserted 26000 rows...
Inserted 28000 rows...
Inserted 30000 rows...
Inserted 32000 rows...
Inserted 34000 rows...
Inserted 36000 rows...
Inserted 38000 rows...
Inserted 40000 rows...
Inserted 42000 rows...
Inserted 44000 rows...
Inserted 46000 rows...
Inserted 48000 rows...
Inserted 50000 rows...
Inserted 52000 rows...
Inserted 54000 rows...
Inserted 56000 rows...
Inserted 58000 rows...
Inserted 60000 rows...
Inserted 62000 rows...
Inserted 64000 rows...
Inserted 66000 rows...
Inserted 68000 rows...
Inserted 70000 rows...
Inserted 72000 rows...
Inserted 74000 rows...
Inserted 76000 rows...
Inserted 78000 rows...
Inserted 80000 rows...


In [15]:
import boto3
import pandas as pd

# AWS Configuration
s3 = boto3.client("s3")
bucket_name = "fraud-detection-data-warehouse"
file_name = "fraud_test_latest_One.csv"

# Save DataFrame as CSV and upload to S3
df.to_csv(file_name, index=False)
s3.upload_file(file_name, bucket_name, file_name)

print(f"File {file_name} uploaded to {bucket_name} bucket in AWS")

File fraud_test_latest_One.csv uploaded to fraud-detection-data-warehouse bucket in AWS
